# Topic 2, Part 2 - CPA on Hardware AES Implementation

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *By now you should have a pretty good understanding of how software implementations of AES are vulnerable to CPA attacks. You might be wondering: are hardware implementations of AES also vulnerable to CPA attacks?*

*In this lab, we'll perform a CPA attack on the hardware AES implementation in the STM32F415. We'll also introduce LASCAR for increased performance when analyzing large datasets.*

**LEARNING OUTCOMES:**
* Understanding how leakage differs between software AES and hardware AES implementations
* Using LASCAR for CPA attacks
* Identifying different leakage points

Capture traces as normal. We'll need to select the HWAES crypto target instead of TINYAES or MBEDTLS. Also we don't need to capture as many traces - the whole AES block will fit in less than 2000 traces. We'll also boost the gain a little bit - HWAES won't result in as big of power spikes:

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'HWAES'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
project = cw.create_project("32bit_AES.cwp", overwrite=True)

In [ ]:
#Capture Traces
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()

traces = []
N = 55000  # Number of traces
scope.adc.samples=2000

scope.gain.db = 38


for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

print(scope.adc.trig_count)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
key_guess = []
xrange = range(len(project.waves[0]))
p.line(xrange, project.waves[0], color="red")
    
show(p)

## Introducing LASCAR

With how many traces we're capturing, analyzing our traces will take a lot of time with ChipWhisperer - Analyzer wasn't designed for performance. It is for this reason that we will be using LASCAR, an open source side channel analysis library with a bigger emphasis on speed than ChipWhisperer Analyzer. Normally, it would take a bit of work to massage ChipWhisperer into the LASCAR format; however, ChipWhisperer has recently integrated some basic LASCAR support, making it easy to combine LASCAR and ChipWhisperer projects! Note that this support is a WIP and not offically documented - the interface can change at any time!

Basic setup is as follows:

In [ ]:
import scared

In [ ]:
%%time
@scared.attack_selection_function
def ahhh(ciphertext, guesses):
    res = np.empty((ciphertext.shape[0], len(guesses), ciphertext.shape[1]), dtype='uint8')
    for i, guess in enumerate(guesses):
        st10 = scared.aes.shift_rows(ciphertext)
        st9 = scared.aes.inv_sub_bytes(np.bitwise_xor(ciphertext, guess))
        res[:,i,:] = np.bitwise_xor(st9, st10)
    return res
a = scared.CPAAttack(selection_function=ahhh,
                    model=scared.HammingWeight(),
                    discriminant=scared.maxabs)
asdfasdf = estraces.read_ths_from_ram(np.array([wave for wave in project.waves], dtype='float64'), 
                                      ciphertext=np.array([textin for textin in project.textouts], dtype='uint8'))
container = scared.Container(asdfasdf)
a.run(container)
print(a.scores.transpose()[0][0xD0])
print(a.scores.transpose()[1][0x14])
for byte in np.argmax(a.scores, axis=0):
    print("{:02X}".format(byte%0xFF))

In [ ]:
'''
Best Guess is 46 (Corr = 0.09238554151310395)
Best Guess is 60 (Corr = 0.038079117460913234)
Best Guess is 23 (Corr = 0.03772818538110263)
Best Guess is 1E (Corr = 0.03737463229030691)
Best Guess is B9 (Corr = 0.09971851051573201)
Best Guess is 1C (Corr = 0.038902506976762)
Best Guess is F1 (Corr = 0.044955933046254996)
Best Guess is 43 (Corr = 0.042020295797275585)
Best Guess is 46 (Corr = 0.09046481610245245)
Best Guess is 8B (Corr = 0.040666373485122594)
Best Guess is AE (Corr = 0.04216082735055769)
Best Guess is E0 (Corr = 0.04003730104245075)
Best Guess is B9 (Corr = 0.09358288771172785)
Best Guess is 1D (Corr = 0.03967158303816735)
Best Guess is BF (Corr = 0.043747171954958884)
Best Guess is 09 (Corr = 0.041499344462391334)
'''

In [ ]:
@scared.attack_selection_function
def ahhh(ciphertext, guesses):
    res = np.empty((ciphertext.shape[0], len(guesses), ciphertext.shape[1]), dtype='uint8')
    for i, guess in enumerate(guesses):
        s = scared.aes.inv_sub_bytes(state=np.bitwise_xor(ciphertext, guess))
        res[:,i,:] = np.bitwise_xor(scared.aes.inv_shift_rows(s), ciphertext)
    return res

In [ ]:
a = scared.CPAAttack(selection_function=ahhh,
                    model=scared.HammingWeight(),
                    discriminant=scared.maxabs)

In [ ]:
asdfasdf = estraces.read_ths_from_ram(np.array([wave for wave in project.waves], dtype='float64'), 
                                      plaintext=np.array([textin for textin in project.textins], dtype='uint8'),
                                      ciphertext=np.array([textout for textout in project.textouts], dtype='uint8'))

In [ ]:
print(np.array([textin for textin in project.textins], dtype='uint8')[0])
print(list(project.textins[0]))

In [ ]:
asdfasdf.ciphertext.shape[1]

In [ ]:
container = scared.Container(asdfasdf, frame=slice(1000,1100))

In [ ]:
a.run(container)

In [ ]:
print(a.scores.transpose()[0][0xD0])

In [ ]:
np.__version__

In [ ]:
!pip install git+https://github.com/Ledger-Donjon/lascar

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
key_guess = []
scores = a.scores.transpose()
print(scores[0])


In [ ]:
!conda install -c conda-forge vispy -y

In [ ]:
!pip install pandas

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
key_guess = []
for i in range(16):
    results = cpa_engines[i].finalize()
    xrange = range(len(results[0xD0]))
    guess = abs(results).max(1).argmax()
    print("Best Guess is {:02X} (Corr = {})".format(guess, abs(results).max()))
    p.line(xrange, results[guess], color="red")
    key_guess.append(guess)
    
show(p)

In [ ]:
import chipwhisperer.common.api.lascar as cw_lascar
from lascar import *
cw_container = cw_lascar.CWContainer(project, project.textouts, start=None, end=None) #optional start and end args set start and end points for analysis
guess_range = range(256)

## Leakage Model

Thus far, we've been exclusively focusing on software AES. Here, each AES operation (shift rows, add round key, mix columns, etc) is implemented using one basic operation (XOR, reads/writes, multiplies, etc.) per clock cycle. With a hardware implementation, it's often possible to not only combine basic operations into a block that can run in a single clock cycle, but also combine multiple AES operations and run them in a single block! For example, the CW305 FPGA board can run each round of AES in a single clock cycle!

Because of this, running a CPA attack on hardware AES is much trickier than on software AES. In software, we found that it was easy to search for the outputs of the s-boxes because these values would need to be loaded from memory onto a high-capacitance data bus. This is not necessarily true for hardware AES, where the output of the s-boxes may be directly fed into the next stage of the algorithm. In general, we may need some more knowledge of the hardware implementation to successfully complete an attack. That being said, if we take a look at a block diagram of AES:

![](https://wiki.newae.com/images/8/8e/AES_Encryption.png)

the last round jumps out for a few reasons:

* It's not far removed from the ciphertext or the plaintext
* It's got an AddRoundKey and a SubBytes, meaning we get a nonlinear addition of the key between the ciphertext and the input of the round
* There's no Mix Columns

Let's make a guess at the implementation and say that it'll do the last round in a single clock cycle and store the input and output in the same memory block. Our reset assumption that allowed us to simply use the Hamming weight instead of the Hamming distance also probably won't be valid here. As such, let's use the Hamming distance between the output and the input of the last round.

ChipWhisperer now includes a few leakage models for use with LASCAR:

In [ ]:
leakage = cw_lascar.lastround_HD_gen

Then, we can actually run the analysis. It should chew through our 15k traces in only a minute or two!

In [ ]:
cpa_engines = [CpaEngine("cpa_%02d" % i, leakage(i), guess_range) for i in range(16)]
session = Session(cw_container, engines=cpa_engines).run(batch_size=50)

Let's print out our results and plot the correlation of our guesses:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
key_guess = []
for i in range(2):
    results = cpa_engines[i].finalize()
    xrange = range(len(results[0xD0]))
    guess = abs(results).max(1).argmax()
    print("Best Guess is {:02X} (Corr = {})".format(guess, abs(results).max()))
    p.line(xrange, results[guess], color="red")
    key_guess.append(guess)
    
show(p)

ChipWhisperer also includes a class to interpret the results of the analysis:

In [ ]:
import chipwhisperer.analyzer as cwa
last_round_key = cwa.aes_funcs.key_schedule_rounds(list(project.keys[0]),0,10)
disp = cw_lascar.LascarDisplay(cpa_engines, last_round_key)
disp.show_pge()

Interestingly, you should see that the attack has worked fairly well for most of the bytes. All of them, in fact, except bytes 0, 4, 8, and 12. Looking the correlation plot, you should see two large spikes instead of one like you might expect. Try focusing the attack on either one of these points by adjusting `start=` and `end=` when making the `cw_container` and try answering the following questions:

* Which spike was our expected leakage actually at (last round state diff)?
* How might you be able to tell that the attack failed for certain bytes at the incorrect leakage point?
* Why might this other spike be occuring?

In [ ]:
scope.dis()
target.dis()